# Full Implementation of Neural Network

#### Import Libraries

In [12]:
import scipy.io
import numpy as np

#### Load weights to verify cost function

In [4]:
mat = scipy.io.loadmat('ex4weights.mat')

#### Retrieving parameters for the input and hidden layers

In [10]:
Theta1 = mat['Theta1']
Theta2 = mat['Theta2']

### Loading the DataSet

In [40]:
data = scipy.io.loadmat('ex4data1.mat')

#### Getting X and y matrices

In [111]:
X = data['X']
y = data['y']

### Unroll parameters

In [36]:
print(np.shape(Theta1), 25*401)
print(np.shape(Theta2), 10*26)

(25, 401) 10025
(10, 26) 260


In [26]:
nn_params = np.concatenate([Theta1.flatten(), Theta2.flatten()])
np.shape(nn_params)

(10285,)

### Rollback

In [39]:
Theta1 = np.reshape(nn_params[0:10025], (-1, 401))
Theta2 = np.reshape(nn_params[10025:10285], (-1, 26))

#### Setup some useful variables

In [50]:
m = np.shape(X)[0] # number of  examples
n = np.shape(X)[1] # number of features
k = 10 # number of labels
J = 0 # Cost function value

## Implementing the cost function

#### Converting y( y(i) -> 1:10) to Y( Y(i) -> 0, 1)

In [117]:
# Y = matrix of where each row(i) 
# represents corresponding X(i) example
# and each column correlates to output's classes
y[y == 0] = 10

Y = np.zeros([m, k])
for i in range(m):
    j = y[i]
    Y[i, j-1] = 1

#### Implementing sigmoid function

In [59]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

## Implementing Forward Propagation

In [123]:
#predictions = sigmoid(np.ones([m, 1]), sigmoid([np.ones([])]))

# Added bias feature to the input layer
a1 = np.concatenate([np.ones([m, 1]), X], axis=1)

# Activation of the hidden layer
z2 = sigmoid(np.matmul(a1, np.transpose(Theta1)))
      
# Added bias feature to the hidden layer
a2 = np.concatenate([np.ones([m, 1]), z2], axis=1)

# Activation of the output layer
z3 = sigmoid(np.matmul(a2, np.transpose(Theta2)))

# The cost function value:
logErrors = np.multiply(Y, np.log(z3)) + np.multiply(np.subtract(1, Y), np.log(np.subtract(1, z3)))
J = -1/m * np.sum(logErrors)

print(f"The cost value is {J}")

The cost value is 0.2876291651613189
